In [2]:
from PIL import Image
from pylab import *
import sys
import numpy as np
import skimage.color
import skimage.io
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [3]:
lemmatizer= nltk.stem.WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
data = pd.read_excel(r"C:\Users\91702\Desktop\Purdue\AUD\Silverado.xlsx")
shape(data)
Description = data["description"]
shape(Description)
Collection_tokenized= []

for i in Description:
    Tokenize = nltk.word_tokenize(i)
    Collection_tokenized.append(Tokenize)
shape(Collection_tokenized)
Lower_List = []
for i in Collection_tokenized:
    Temp_list = []
    for j in i:
        Temp_list.append(j.lower())
    Lower_List.append(Temp_list)
Lemmatized_list = []
for j in Lower_List:
    Lemmatized_list_temp = [lemmatizer.lemmatize(i) for i in j if i.isalpha()]
    Lemmatized_list.append(Lemmatized_list_temp)
Stopwords_removed = []
for i in Lemmatized_list :
    Temp_list = [j for j in i if not j in stopwords.words('english')]
    Stopwords_removed.append(Temp_list)
shape(Stopwords_removed)
Td_idf_pre = []
for i in Stopwords_removed:
    Temp_sentence = ""
    for j in i:
         Temp_sentence = Temp_sentence + " " + j
    Td_idf_pre.append(Temp_sentence)
shape(Td_idf_pre)
Vectorizer = TfidfVectorizer(stop_words="english")
X = Vectorizer.fit_transform(Td_idf_pre)
true_k = 4
model1 = KMeans(n_clusters=true_k)
model1.fit(X)
model1.cluster_centers_
model1.fit_transform(X)
model1clusters = model1.labels_.tolist()
print(model1clusters)
order_centroids = model1.cluster_centers_.argsort()[:, ::-1]
terms = Vectorizer.get_feature_names()
for ind in order_centroids[1, :20]:
    print(terms[ind]) 
for ind in order_centroids[0, :20]:
    print(terms[ind])
shape(model1clusters)
data["clusters"] = model1clusters
data["New_desc"] = Td_idf_pre
data.to_csv("Vehicle new.csv")
data
import numpy as np
import nltk
from nltk.corpus import stopwords
from numpy import array
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')




df = pd.read_csv('vehicles.csv')


df1 = df[['id', 'url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'type', 'paint_color', 'description', 'state']]

df2 = df[['id', 'url', 'price', 'description','model','manufacturer']]


df2.dropna(inplace=True)

import matplotlib.pyplot as plt
%matplotlib inline


df2[df2['model']== 'f-150'].sort_values(by=['price'])['price'].plot(kind='hist',bins = 40)

df2[df2['model']== 'f-150'].sort_values(by=['price'])[['price','description']]

model = df2[df2['model']== 'f-150']

model.drop_duplicates(subset = 'description',inplace=True)



#model.to_csv('model_original.csv')

new = pd.read_csv('model_original.csv')

df3 = new

# FUNCTIONS

#STEP 1 - Tokenized each review in the collection
def TOKEN(x):
    z = nltk.word_tokenize(x)
    return z
 
#STEP 2 - lemmatized all the words for each review
lemmatizer= nltk.stem.WordNetLemmatizer()
def LEMMA(x):
    lemmatized_token_d2 = [lemmatizer.lemmatize(token) for token in x if token.isalpha()]
    return lemmatized_token_d2

#STEP 2 lowered all the words
def LOWER(s):
    a = [x.lower() for x in s]
        
    return a
#STEP 3 - removed Stop Words
def STOP(x):
    stop_words_removed= [token for token in x if not token in 
                     stopwords.words('english') if token.isalpha()]
    return stop_words_removed

def JOIN(x):
    q = ' '.join(str(e) for e in x) 
    return q


df3.drop_duplicates(subset='description',inplace = True)


df3['tokenised'] = df3['description'].apply(TOKEN)
df3['Lametise'] = df3['tokenised'].apply(LEMMA)
df3['Lower'] = df3['Lametise'].apply(LOWER)
df3['Stop'] = df3['Lower'].apply(STOP)
df3['Stop_new'] = df3['Stop'].apply(JOIN)

def POS(x):
    q = nltk.pos_tag(x) 
    return q


list1 = list(df3['Stop'])

Abbreviation = [
'NN',
'NNS',
'NNP',
'NNPS',
'PDT',
'POS',
'RB',
'RBR',
'RBS',
'VB',
'VBG',
'VBD',
'VBN',
'VBP',
'VBZ',
'WRB']


fin_list = []
for y in range (len(list1)):
    list2 =[]
    for x in range(len(list1[y])):
        if POS(list1[y])[x][1] in Abbreviation :
            list2.append(POS(list1[y])[x][0])
        
    fin_list.append(list2)
    



df3[['id','price', 'Spam', 'description']]

from sklearn.model_selection import train_test_split

X = df3['Stop_new']  
y = df3['Spam']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) 
X_train_tfidf.shape


from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

text_clf.fit(X_train, y_train)  

# Form a prediction set
predictions = text_clf.predict(X_test)

# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

# Print a classification report
print(metrics.classification_report(y_test,predictions))

stopwords_rem = list(df3['Stop_new'])

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=5)
vectorizer.fit(stopwords_rem)
cv = vectorizer.transform(stopwords_rem)

terms = vectorizer.get_feature_names()	

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(3).fit(cv)
topic_list = []
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    topic_list.append(" ".join([terms[i] for i in topic.argsort()[:-6-1:-1]]))
    print(topic_list[-1])
lda.transform(cv[1])
#print(lda.components_)

df3['Topic'] = topic_dist.argmax(axis=1)

df3[df3['Topic']==1]['description'].head()

df3[df3['Topic']==0]['description'].head()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([terms[i] for i in topic.argsort()[:-15-1:-1]]))

spam = df3[df3['Spam']==0]

list1 = list(spam['Stop_new'])


from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=5)

vectorizer.fit(list1)
v3 = vectorizer.transform(list1)


Feautur_name = vectorizer.get_feature_names()
dense = v3.todense()
dense_list = dense.tolist()

X = array(dense_list)
y = spam['price']

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Dense(5,activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(3,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

model.fit(x=X_train,y=y_train.values,
          validation_data=(X_test,y_test.values),
          batch_size=128,epochs=500)

